In [19]:
import pandas as pd
import numpy as np
from Project.Database import Db
from Project._03FeatureSelection.NZERTF_dataframe_redundancy_functions import find_average_power_consumption_per_minute
from Project._05InferKnowledgeOfRules.infer_rules_functions import SE_time_df, json_to_dataframe

In [20]:
app_stats = SE_time_df(json_to_dataframe(year=2, level=1, with_redundancy=False))

In [155]:
app_stats['Load_StatusPlugLoadIron']

,TotalAbsSupport,AbsSupport,EventCount,ExternalUtility,RelSupport,TimeAssociation,Flexibility,Lifespan
0,0,0,105,0.287671,0.000000,0,0.125,24
1,0,0,105,0.287671,0.000000,0,0.125,24
2,0,0,105,0.287671,0.000000,0,0.125,24
3,0,0,105,0.287671,0.000000,0,0.125,24
4,0,0,105,0.287671,0.000000,0,0.125,24
5,0,0,105,0.287671,0.000000,0,0.125,24
6,0,0,105,0.287671,0.000000,0,0.125,24
7,0,0,105,0.287671,0.000000,0,0.125,24
8,0,0,105,0.287671,0.000000,0,0.125,24
9,0,0,105,0.287671,0.000000,0,0.125,24


In [22]:
average_power_consumption_per_minute_df = find_average_power_consumption_per_minute()
average_power_consumption_per_minute_df

,PowerConsumption(Wm)
Load_StatusApplianceCooktop,1496.0
Load_StatusApplianceDishwasher,282.0
Load_StatusApplianceOven,771.0
Load_StatusApplianceRangeHood,78.0
Load_StatusBA1Lights,48.0
Load_StatusBA2Lights,11.0
Load_StatusBR2Lights,40.0
Load_StatusBR3Lights,33.0
Load_StatusBR4Lights,27.0
Load_StatusDRLights,38.0


In [23]:
average_power_consumption_per_minute_df.loc[~average_power_consumption_per_minute_df.index.isin(app_stats.keys())]

,PowerConsumption(Wm)
Load_StatusBR4Lights,27.0
Load_StatusEntryHallLights,16.0
Load_StatusLRLights1,18.0
Load_StatusLatentload,45.0
Load_StatusPlugLoadHairDryerCurlIron,1515.0


In [24]:
matrix_dict={}
for key, value in app_stats.items():
    lifespan = value['Lifespan'][0]
    hour_slots = int(np.ceil(lifespan/60))

    average_power_consumption_per_minute = average_power_consumption_per_minute_df.loc[lambda self: self.index == key].values[0][0]

    hour_list = [0 for n in range(24)]
    for i in range(hour_slots):
        hour_list[i] = round(average_power_consumption_per_minute*lifespan/60_000/hour_slots, 4)

    temp_dict={}
    for n in range(24+1-hour_slots):
        rotate_list = [hour_list[(i - n) % len(hour_list)] for i, x in enumerate(hour_list)]
        temp_dict.update({n: rotate_list})

    matrix_dict.update({key: pd.DataFrame(temp_dict).loc[:,lambda self: self.sum(0) == self.T.dot(app_stats[key]['TimeAssociation'])]})

matrix_dict

{'Load_StatusSensHeatPrntAUP':          0       1       2
 0   0.0642  0.0000  0.0000
 1   0.0642  0.0642  0.0000
 2   0.0642  0.0642  0.0642
 3   0.0642  0.0642  0.0642
 4   0.0642  0.0642  0.0642
 5   0.0642  0.0642  0.0642
 6   0.0642  0.0642  0.0642
 7   0.0000  0.0642  0.0642
 8   0.0000  0.0000  0.0642
 9   0.0000  0.0000  0.0000
 10  0.0000  0.0000  0.0000
 11  0.0000  0.0000  0.0000
 12  0.0000  0.0000  0.0000
 13  0.0000  0.0000  0.0000
 14  0.0000  0.0000  0.0000
 15  0.0000  0.0000  0.0000
 16  0.0000  0.0000  0.0000
 17  0.0000  0.0000  0.0000
 18  0.0000  0.0000  0.0000
 19  0.0000  0.0000  0.0000
 20  0.0000  0.0000  0.0000
 21  0.0000  0.0000  0.0000
 22  0.0000  0.0000  0.0000
 23  0.0000  0.0000  0.0000,
 'Load_StatusSensHeatPrntBUP':         0      1      2      3      4      5
 0   0.068  0.000  0.000  0.000  0.000  0.000
 1   0.068  0.068  0.000  0.000  0.000  0.000
 2   0.068  0.068  0.068  0.000  0.000  0.000
 3   0.068  0.068  0.068  0.068  0.000  0.000
 4   0.06

In [25]:
production = Db.load_data(consumption=False, production=True, year=2)

In [26]:
production_vectors=production.groupby([production.index.strftime('%Y-%m-%d'), production.index.hour]).sum()['CO2(Grams)/kWh']

In [27]:
production_vectors

2015-02-01  0     734.303195
            1     752.404834
            2     737.093200
            3     733.296185
            4     731.040668
                     ...    
2016-01-31  19    684.508145
            20    697.111722
            21    544.997583
            22    420.530000
            23    420.530000
Name: CO2(Grams)/kWh, Length: 8760, dtype: float64

In [28]:
production_vectors.loc[production_vectors.index.get_level_values(level=0).unique().tolist()[1]]

0     520.166336
1     687.216419
2     377.188378
3     444.096050
4     618.120622
5     714.704375
6     632.085625
7     793.932749
8     812.025157
9     748.803812
10    744.904973
11    739.975500
12    710.935000
13    686.380085
14    721.358380
15    710.935000
16    787.369596
17    741.218715
18    765.061067
19    774.064800
20    746.725762
21    769.016000
22    892.073655
23    837.505000
Name: CO2(Grams)/kWh, dtype: float64

In [29]:
pd.to_datetime(production_vectors.index.get_level_values(level=0).unique(), format='%Y-%m-%d').weekday.map(lambda self: {0: 'Monday',
                   1: 'Tuesday',
                   2: 'Wednesday',
                   3: 'Thursday',
                   4: 'Friday',
                   5: 'Saturday',
                   6: 'Sunday',}[self])

Index(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'Monday', 'Tuesday',
       ...
       'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday'],
      dtype='object', length=365)

In [30]:
def production_vector(day_number: int=0):
    return production_vectors.loc[production_vectors.index.get_level_values(level=0).unique().tolist()[day_number]]

In [177]:
def find_min_hour(appliance: str, day_number: int, number_of_events: int):
    if number_of_events == 1:
        return matrix_dict[appliance].T.dot(production_vector(day_number))[lambda self: self == self.min()]
    else:
        events = matrix_dict[appliance].T.dot(production_vector(day_number))[lambda self: self == self.min()]

        condition = (lambda df:
                     (df.sum(0) != df.T.dot(np.where(pd.DataFrame(df[events.index.tolist()]).max(1) > 0, 1, 0))))
        condition2 = (lambda df, events_index:
                      df.loc[:, (df.loc[events_index] == 0)])

        return matrix_dict[appliance].loc[:,
               (lambda self:
                condition2(self, events.index.tolist()))]

        # for _ in range(number_of_events-1):
        #     events = pd.concat(objs=(
        #         events,
        #         matrix_dict[appliance].loc[:,lambda self: self.sum(0) != self.T.dot(pd.DataFrame(matrix_dict[appliance][events.index.tolist()]).max(1))]
        #     )
        #     )
        # return events

In [181]:
matrix_dict['Load_StatusApplianceDishwasher'].loc[matrix_dict['Load_StatusApplianceDishwasher'].loc[21]==0]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [179]:
find_min_hour('Load_StatusApplianceDishwasher', day_number=0, number_of_events=1)

21    319.382694
dtype: float64

In [180]:
find_min_hour('Load_StatusApplianceDishwasher', day_number=0, number_of_events=2)

ValueError: Cannot index with multidimensional key

In [32]:
year2, meta = Db.load_data(consumption=True, year=2, meta=True, hourly=False)

In [33]:
meta.loc['Load_StatusPlugLoadIron']

Subsystem                                                           Loads
Measurement_Location                                        MasterBedroom
Parameter                                                    Status_OnOff
Description             Number to indicate whether iron is activated (...
Units                                                        BinaryStatus
Aggregation_Method                                                Average
min_value                                                             0.0
max_value                                                             1.0
Standby_Power                                                         NaN
Measurement_Floor                                                2ndFloor
Load_Match                                    Load_MBRPlugLoadsPowerUsage
Status_Match                                                          NaN
Elec_Match                                        Elec_PowerPlugsInstMBRA
Consumer_Match                        

In [34]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        # (~self.index.str.contains("Child")) &
        # (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload"))  #&
    # (~self.index.str.contains("Load_StatusClothesWasher")) &
    # (~self.index.str.contains("Load_StatusDryerPowerTotal"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

In [35]:
minute_appliances_status = year2[["Timestamp"] + appliances_statusOnOff].copy()

In [36]:
movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher', 'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

In [37]:
psns=['Load_StatusSensHeatChildADOWN',
'Load_StatusSensHeatChildAUP',
'Load_StatusSensHeatChildBDOWN',
'Load_StatusSensHeatChildBUP',
'Load_StatusSensHeatPrntADOWN',
'Load_StatusSensHeatPrntAUP',
'Load_StatusSensHeatPrntBDOWN']

In [38]:
minute_appliances_status.groupby(minute_appliances_status['Timestamp'].dt.strftime('%Y-%m-%d')).max()[movable_appliances]

,Load_StatusApplianceDishwasher,Load_StatusPlugLoadVacuum,Load_StatusClothesWasher,Load_StatusDryerPowerTotal,Load_StatusPlugLoadIron
Timestamp,,,,,
2015-02-01,1,0,1,1,1
2015-02-02,1,0,0,0,0
2015-02-03,0,0,0,0,0
2015-02-04,1,0,1,1,0
2015-02-05,0,0,0,0,0
...,...,...,...,...,...
2016-01-27,1,0,1,1,0
2016-01-28,0,0,0,0,0
2016-01-29,1,0,0,0,0


In [39]:
level1_json_df = json_to_dataframe(year=2, level=1, exclude_follows=True, with_redundancy=True)

In [40]:
level1_movable_appliances = level1_json_df.loc[lambda self: self['pattern'].isin(movable_appliances)].set_index('pattern')

In [41]:
level1_movable_appliances

,supp,conf,time,rule,multi_floor,floor
pattern,,,,,,
Load_StatusClothesWasher,0.427397,1.0,"{'0': [['2015-02-01 11:00:49', '2015-02-01 11:...",app,False,1stFloor
Load_StatusDryerPowerTotal,0.432877,1.0,"{'0': [['2015-02-01 12:30:49', '2015-02-01 13:...",app,False,1stFloor
Load_StatusPlugLoadIron,0.287671,1.0,"{'0': [['2015-02-01 14:15:49', '2015-02-01 14:...",app,False,2ndFloor
Load_StatusApplianceDishwasher,0.712329,1.0,"{'0': [['2015-02-01 20:27:48', '2015-02-01 22:...",app,False,1stFloor
Load_StatusPlugLoadVacuum,0.142466,1.0,"{'6': [['2015-02-07 17:30:00', '2015-02-07 19:...",app,False,1stFloor


In [42]:
def find_event_count(appliance: str, day_number: int=0):
    try:
        return len(level1_movable_appliances.loc[appliance]['time'][f'{day_number}'])
    except:
        return 0

In [43]:
find_event_count(appliance='Load_StatusPlugLoadIron', day_number=0)

1

In [51]:
find_min_hour(appliance='Load_StatusPlugLoadIron', day_number=0)

14    354.233999
dtype: float64

In [112]:
possible_hours = find_min_hour(appliance='Load_StatusClothesWasher', day_number=0)
possible_hours



11    119.528573
12    117.446462
14    112.278313
15    121.601684
18    116.494150
19     97.198692
20    106.625121
21     88.847479
dtype: float64

In [95]:
optim_dict={}
for apps in movable_appliances:
    appended_data = []
    for i in range(300):
        event_count = find_event_count(appliance=apps, day_number=i)
        temporary_df=pd.DataFrame(find_min_hour(appliance=apps, day_number=i), columns=['emissions'])
        temporary_df['start_hour'] = temporary_df.index
        temporary_df['day'] = i
        temporary_df['lifespan'] = app_stats[apps]['Lifespan'][0]
        appended_data.append(temporary_df)
    appended_data = pd.concat(appended_data)
    optim_dict.update({apps: appended_data})

optim_dict

{'Load_StatusApplianceDishwasher':      emissions  start_hour  day  lifespan
 21  319.382694          21    0       133
 20  501.788733          20    1       133
 21  449.983919          21    2       133
 20  500.578975          20    3       133
 20  401.119532          20    4       133
 ..         ...         ...  ...       ...
 21  441.353396          21  295       133
 21  442.343842          21  296       133
 21  219.519463          21  297       133
 20  212.366331          20  298       133
 21  290.839469          21  299       133
 
 [304 rows x 4 columns],
 'Load_StatusPlugLoadVacuum':      emissions  start_hour  day  lifespan
 18  697.606233          18    0       111
 17  812.336686          17    1       111
 18  785.830843          18    2       111
 17  776.409511          17    3       111
 17  750.537270          17    4       111
 ..         ...         ...  ...       ...
 17  730.340628          17  295       111
 18  755.156817          18  296       111
 17  58